In [2]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

In [3]:
df = pd.read_csv('dataset_pollution_timeseries.csv')
df = df.iloc[:, 1:]

- dew: Dew Point
- temp: Temperature
- press: Pressure
- wnd_spd: Cumulated wind speed
- snow: Cumulated hours of snow
- rain: Cumulated hours of rain
- pollution: 1 (yes) or 0 (no), threshold=80

In [4]:
y = df[['timestamp', 'pollution']].groupby('timestamp').max()
y.value_counts()

pollution
0            947
1            878
dtype: int64

In [5]:
df = df.set_index('timestamp')
X = df.drop(['pollution', 'hour'], axis=1)
X.head()

,dew,temp,press,wnd_spd,snow,rain
timestamp,,,,,,
2010-01-02,-16,-4.0,1020.0,1.79,0,0
2010-01-02,-15,-4.0,1020.0,2.68,0,0
2010-01-02,-11,-5.0,1021.0,3.57,0,0
2010-01-02,-7,-5.0,1022.0,5.36,1,0
2010-01-02,-7,-5.0,1022.0,6.25,2,0


In [6]:
train_size = int(len(X.index) * 0.8)
X_train = X.iloc[:train_size, :]
X_test = X.iloc[train_size:, :]

In [7]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# 1) LSTM

In [8]:
X_train_lstm = X_train.reshape(int(X_train.shape[0]/24), -1, X_train.shape[1])		# shape: 1460 days (4 years), 24 hours, 7 features
y_train_lstm = np.array(y)[:int(X_train_lstm.shape[0])]

X_test_lstm = X_test.reshape(int(X_test.shape[0]/24), -1, X_test.shape[1])			# shape: 365 days (1 year), 24 hours, 7 features
y_test_lstm = np.array(y)[int(X_train_lstm.shape[0]):]

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rnn = Sequential()
rnn.add(LSTM(units=100, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
rnn.add(LSTM(units=100, return_sequences=False))
rnn.add(Dense(20, activation='relu'))
rnn.add(Dense(1, activation='sigmoid'))
rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_data=(X_test_lstm, y_test_lstm), verbose=1)

Epoch 1/100
46/46 [==============================] - 5s 46ms/step - loss: 0.6172 - accuracy: 0.6370 - val_loss: 0.5966 - val_accuracy: 0.6603
Epoch 2/100
46/46 [==============================] - 1s 24ms/step - loss: 0.5382 - accuracy: 0.7212 - val_loss: 0.6348 - val_accuracy: 0.6247
Epoch 3/100
46/46 [==============================] - 1s 21ms/step - loss: 0.5263 - accuracy: 0.7349 - val_loss: 0.5712 - val_accuracy: 0.6795
Epoch 4/100
46/46 [==============================] - 1s 22ms/step - loss: 0.5174 - accuracy: 0.7315 - val_loss: 0.5868 - val_accuracy: 0.6959
Epoch 5/100
46/46 [==============================] - 1s 22ms/step - loss: 0.5261 - accuracy: 0.7301 - val_loss: 0.5573 - val_accuracy: 0.6932
Epoch 6/100
46/46 [==============================] - 1s 23ms/step - loss: 0.5155 - accuracy: 0.7397 - val_loss: 0.5373 - val_accuracy: 0.7425
Epoch 7/100
46/46 [==============================] - 1s 25ms/step - loss: 0.5184 - accuracy: 0.7384 - val_loss: 0.5415 - val_accuracy: 0.7068
Epoch 

46/46 [==============================] - 1s 25ms/step - loss: 0.4455 - accuracy: 0.7897 - val_loss: 0.4815 - val_accuracy: 0.7726
Epoch 59/100
46/46 [==============================] - 1s 28ms/step - loss: 0.4419 - accuracy: 0.7938 - val_loss: 0.5071 - val_accuracy: 0.7699
Epoch 60/100
46/46 [==============================] - 1s 27ms/step - loss: 0.4424 - accuracy: 0.7932 - val_loss: 0.5073 - val_accuracy: 0.7644
Epoch 61/100
46/46 [==============================] - 1s 26ms/step - loss: 0.4319 - accuracy: 0.7911 - val_loss: 0.5156 - val_accuracy: 0.7753
Epoch 62/100
46/46 [==============================] - 1s 30ms/step - loss: 0.4477 - accuracy: 0.7863 - val_loss: 0.5160 - val_accuracy: 0.7260
Epoch 63/100
46/46 [==============================] - 2s 37ms/step - loss: 0.4429 - accuracy: 0.7973 - val_loss: 0.4879 - val_accuracy: 0.7671
Epoch 64/100
46/46 [==============================] - 1s 32ms/step - loss: 0.4545 - accuracy: 0.7760 - val_loss: 0.5053 - val_accuracy: 0.7671
Epoch 65/100

In [11]:
from sklearn.metrics import roc_auc_score

y_pred = rnn.predict(X_test_lstm)
auc = roc_auc_score(y_test_lstm, y_pred)
print('AUC: ', auc)

AUC:  0.827405668893302


# 2) CNN (over features)

In [12]:
X_train_cnn = X_train.reshape(int(X_train.shape[0]/24), -1, X_train.shape[1], 1)		# shape: 1460 days (4 years), 24 hours, 6 features
y_train_cnn = np.array(y)[:int(X_train_cnn.shape[0])]

X_test_cnn = X_test.reshape(int(X_test.shape[0]/24), -1, X_test.shape[1], 1)			# shape: 365 days (1 year), 24 hours, 6 features
y_test_cnn = np.array(y)[int(X_train_cnn.shape[0]):]

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Flatten

cnn = Sequential()

conv_layer = Conv2D(filters=32, kernel_size=(1, X_train_cnn.shape[2]), activation='relu', input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2], 1))
cnn.add(conv_layer)
print(conv_layer.output_shape)

flatten_layer = Flatten()
cnn.add(flatten_layer)
print(flatten_layer.output_shape)

cnn.add(Dense(20, activation='relu'))
cnn.add(Dense(units=1))
cnn.compile(optimizer='adam', loss='mean_absolute_error')
cnn.fit(X_train_cnn, y_train_cnn, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_cnn, y_test_cnn))

(None, 24, 1, 32)
(None, 768)
Epoch 1/100
46/46 [==============================] - 0s 3ms/step - loss: 0.4473 - val_loss: 0.4815
Epoch 2/100
46/46 [==============================] - 0s 1ms/step - loss: 0.4063 - val_loss: 0.4689
Epoch 3/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3854 - val_loss: 0.4177
Epoch 4/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3478 - val_loss: 0.3834
Epoch 5/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3364 - val_loss: 0.3934
Epoch 6/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3277 - val_loss: 0.3692
Epoch 7/100
46/46 [==============================] - 0s 2ms/step - loss: 0.3211 - val_loss: 0.3843
Epoch 8/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3175 - val_loss: 0.3863
Epoch 9/100
46/46 [==============================] - 0s 2ms/step - loss: 0.3142 - val_loss: 0.3637
Epoch 10/100
46/46 [==============================] - 0s 1ms/step - loss: 0.324

46/46 [==============================] - 0s 2ms/step - loss: 0.2429 - val_loss: 0.2980
Epoch 83/100
46/46 [==============================] - 0s 2ms/step - loss: 0.2471 - val_loss: 0.2987
Epoch 84/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2402 - val_loss: 0.2925
Epoch 85/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2411 - val_loss: 0.2956
Epoch 86/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2425 - val_loss: 0.2958
Epoch 87/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2424 - val_loss: 0.2928
Epoch 88/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2403 - val_loss: 0.2988
Epoch 89/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2384 - val_loss: 0.2914
Epoch 90/100
46/46 [==============================] - 0s 2ms/step - loss: 0.2385 - val_loss: 0.2909
Epoch 91/100
46/46 [==============================] - 0s 2ms/step - loss: 0.2387 - val_loss: 0.2906
Epoch 92/100


In [15]:
y_pred = cnn.predict(X_test_cnn)
auc = roc_auc_score(y_test_cnn, y_pred)
print('AUC: ', auc)

AUC:  0.8248781368478064


# 2) CNN (over time)

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Flatten

cnn = Sequential()

conv_layer = Conv2D(filters=32, kernel_size=(24, 1), activation='relu', input_shape=(X_train_cnn.shape[1], X_train_cnn.shape[2], 1))
cnn.add(conv_layer)
print(conv_layer.output_shape)

flatten_layer = Flatten()
cnn.add(flatten_layer)
print(flatten_layer.output_shape)

cnn.add(Dense(20, activation='relu'))
cnn.add(Dense(units=1))
cnn.compile(optimizer='adam', loss='mean_absolute_error')
cnn.fit(X_train_cnn, y_train_cnn, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_cnn, y_test_cnn))

(None, 1, 6, 32)
(None, 192)
Epoch 1/100
46/46 [==============================] - 0s 3ms/step - loss: 0.4612 - val_loss: 0.4798
Epoch 2/100
46/46 [==============================] - 0s 1ms/step - loss: 0.4167 - val_loss: 0.4711
Epoch 3/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3949 - val_loss: 0.4435
Epoch 4/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3737 - val_loss: 0.4247
Epoch 5/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3624 - val_loss: 0.4246
Epoch 6/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3533 - val_loss: 0.4095
Epoch 7/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3543 - val_loss: 0.4117
Epoch 8/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3442 - val_loss: 0.4021
Epoch 9/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3416 - val_loss: 0.4079
Epoch 10/100
46/46 [==============================] - 0s 1ms/step - loss: 0.3403

46/46 [==============================] - 0s 1ms/step - loss: 0.2611 - val_loss: 0.3363
Epoch 83/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2625 - val_loss: 0.3248
Epoch 84/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2648 - val_loss: 0.3261
Epoch 85/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2563 - val_loss: 0.3222
Epoch 86/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2574 - val_loss: 0.3156
Epoch 87/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2572 - val_loss: 0.3309
Epoch 88/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2588 - val_loss: 0.3212
Epoch 89/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2582 - val_loss: 0.3110
Epoch 90/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2544 - val_loss: 0.3142
Epoch 91/100
46/46 [==============================] - 0s 1ms/step - loss: 0.2574 - val_loss: 0.3204
Epoch 92/100


In [17]:
y_pred = cnn.predict(X_test_cnn)
auc = roc_auc_score(y_test_cnn, y_pred)
print('AUC: ', auc)

AUC:  0.8253896611903473


# ANN (flattened features)

In [18]:
X = pd.concat([X, df['hour']], axis=1)

In [19]:
X['g'] = X.groupby('hour')['hour'].cumcount()
X = X.set_index(['g','hour']).stack().unstack([1, 2])

In [20]:
train_size = int(len(X.index) * 0.8)
X_train_ann = X.iloc[:train_size, :]
X_test_ann = X.iloc[train_size:, :]

In [21]:
sc = MinMaxScaler()
X_train_ann = sc.fit_transform(X_train_ann)
X_test_ann = sc.transform(X_test_ann)

y_train_ann = np.array(y)[:int(X_train_ann.shape[0])]
y_test_ann = np.array(y)[int(X_train_ann.shape[0]):]

In [22]:
import tensorflow as tf

ann = tf.keras.models.Sequential()
ann.add(tf.keras.Input(shape=(144,)))
ann.add(tf.keras.layers.Dense(units = 90, activation='relu'))
ann.add(tf.keras.layers.Dense(units = 30, activation='relu'))
ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train_ann, y_train_ann, batch_size=32, epochs=500, verbose=1, validation_data=(X_test_ann, y_test_ann))

Epoch 1/500
46/46 [==============================] - 1s 3ms/step - loss: 0.6541 - accuracy: 0.6021 - val_loss: 0.6499 - val_accuracy: 0.5671
Epoch 2/500
46/46 [==============================] - 0s 1ms/step - loss: 0.5935 - accuracy: 0.6911 - val_loss: 0.6077 - val_accuracy: 0.6685
Epoch 3/500
46/46 [==============================] - 0s 1ms/step - loss: 0.5516 - accuracy: 0.7260 - val_loss: 0.6116 - val_accuracy: 0.6247
Epoch 4/500
46/46 [==============================] - 0s 2ms/step - loss: 0.5349 - accuracy: 0.7315 - val_loss: 0.5708 - val_accuracy: 0.6986
Epoch 5/500
46/46 [==============================] - 0s 1ms/step - loss: 0.5173 - accuracy: 0.7418 - val_loss: 0.5690 - val_accuracy: 0.6712
Epoch 6/500
46/46 [==============================] - 0s 1ms/step - loss: 0.5095 - accuracy: 0.7486 - val_loss: 0.5682 - val_accuracy: 0.6932
Epoch 7/500
46/46 [==============================] - 0s 1ms/step - loss: 0.5111 - accuracy: 0.7274 - val_loss: 0.5511 - val_accuracy: 0.7041
Epoch 8/500
4

Epoch 59/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4148 - accuracy: 0.8048 - val_loss: 0.5406 - val_accuracy: 0.7671
Epoch 60/500
46/46 [==============================] - 0s 2ms/step - loss: 0.4165 - accuracy: 0.8055 - val_loss: 0.5321 - val_accuracy: 0.7726
Epoch 61/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4111 - accuracy: 0.8075 - val_loss: 0.5261 - val_accuracy: 0.7507
Epoch 62/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4007 - accuracy: 0.8144 - val_loss: 0.5521 - val_accuracy: 0.6904
Epoch 63/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4104 - accuracy: 0.8089 - val_loss: 0.5572 - val_accuracy: 0.7479
Epoch 64/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8123 - val_loss: 0.5322 - val_accuracy: 0.7534
Epoch 65/500
46/46 [==============================] - 0s 1ms/step - loss: 0.4037 - accuracy: 0.8110 - val_loss: 0.5745 - val_accuracy: 0.6822
Epoch 

46/46 [==============================] - 0s 2ms/step - loss: 0.3759 - accuracy: 0.8171 - val_loss: 0.5545 - val_accuracy: 0.7397
Epoch 117/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3737 - accuracy: 0.8253 - val_loss: 0.5532 - val_accuracy: 0.7342
Epoch 118/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3616 - accuracy: 0.8322 - val_loss: 0.5449 - val_accuracy: 0.7233
Epoch 119/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3678 - accuracy: 0.8267 - val_loss: 0.5183 - val_accuracy: 0.7726
Epoch 120/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3716 - accuracy: 0.8315 - val_loss: 0.5495 - val_accuracy: 0.7534
Epoch 121/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3628 - accuracy: 0.8308 - val_loss: 0.5529 - val_accuracy: 0.7315
Epoch 122/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3578 - accuracy: 0.8356 - val_loss: 0.5723 - val_accuracy: 0.7178
Epoch 123/500

46/46 [==============================] - 0s 1ms/step - loss: 0.3418 - accuracy: 0.8377 - val_loss: 0.5964 - val_accuracy: 0.7151
Epoch 174/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3291 - accuracy: 0.8534 - val_loss: 0.5813 - val_accuracy: 0.7315
Epoch 175/500
46/46 [==============================] - 0s 2ms/step - loss: 0.3232 - accuracy: 0.8527 - val_loss: 0.5494 - val_accuracy: 0.7562
Epoch 176/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3264 - accuracy: 0.8548 - val_loss: 0.5553 - val_accuracy: 0.7781
Epoch 177/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3519 - accuracy: 0.8322 - val_loss: 0.5844 - val_accuracy: 0.7425
Epoch 178/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3313 - accuracy: 0.8507 - val_loss: 0.5735 - val_accuracy: 0.7534
Epoch 179/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3262 - accuracy: 0.8555 - val_loss: 0.5668 - val_accuracy: 0.7397
Epoch 180/500

46/46 [==============================] - 0s 1ms/step - loss: 0.2906 - accuracy: 0.8664 - val_loss: 0.5704 - val_accuracy: 0.7890
Epoch 231/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2942 - accuracy: 0.8664 - val_loss: 0.6414 - val_accuracy: 0.7342
Epoch 232/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3337 - accuracy: 0.8432 - val_loss: 0.6457 - val_accuracy: 0.7397
Epoch 233/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3291 - accuracy: 0.8356 - val_loss: 0.5884 - val_accuracy: 0.7644
Epoch 234/500
46/46 [==============================] - 0s 1ms/step - loss: 0.3016 - accuracy: 0.8630 - val_loss: 0.6285 - val_accuracy: 0.7315
Epoch 235/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2977 - accuracy: 0.8644 - val_loss: 0.6124 - val_accuracy: 0.7507
Epoch 236/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2927 - accuracy: 0.8719 - val_loss: 0.6373 - val_accuracy: 0.7342
Epoch 237/500

46/46 [==============================] - 0s 1ms/step - loss: 0.2634 - accuracy: 0.8815 - val_loss: 0.7820 - val_accuracy: 0.7233
Epoch 288/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2695 - accuracy: 0.8760 - val_loss: 0.7357 - val_accuracy: 0.7370
Epoch 289/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2646 - accuracy: 0.8781 - val_loss: 0.6425 - val_accuracy: 0.7562
Epoch 290/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2895 - accuracy: 0.8712 - val_loss: 0.7248 - val_accuracy: 0.7562
Epoch 291/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2648 - accuracy: 0.8829 - val_loss: 0.7897 - val_accuracy: 0.7068
Epoch 292/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2602 - accuracy: 0.8836 - val_loss: 0.7208 - val_accuracy: 0.7479
Epoch 293/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2562 - accuracy: 0.8884 - val_loss: 0.6963 - val_accuracy: 0.7562
Epoch 294/500

46/46 [==============================] - 0s 1ms/step - loss: 0.2322 - accuracy: 0.9014 - val_loss: 0.9720 - val_accuracy: 0.6904
Epoch 345/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2452 - accuracy: 0.8979 - val_loss: 0.7401 - val_accuracy: 0.7699
Epoch 346/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2393 - accuracy: 0.8952 - val_loss: 0.7716 - val_accuracy: 0.7562
Epoch 347/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2385 - accuracy: 0.8897 - val_loss: 0.8020 - val_accuracy: 0.7233
Epoch 348/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2492 - accuracy: 0.8884 - val_loss: 0.7198 - val_accuracy: 0.7562
Epoch 349/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2388 - accuracy: 0.8897 - val_loss: 0.7882 - val_accuracy: 0.7562
Epoch 350/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2388 - accuracy: 0.8897 - val_loss: 0.8683 - val_accuracy: 0.7014
Epoch 351/500

46/46 [==============================] - 0s 1ms/step - loss: 0.2157 - accuracy: 0.9007 - val_loss: 0.9338 - val_accuracy: 0.7342
Epoch 402/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2166 - accuracy: 0.9075 - val_loss: 0.8315 - val_accuracy: 0.7562
Epoch 403/500
46/46 [==============================] - 0s 2ms/step - loss: 0.2225 - accuracy: 0.8938 - val_loss: 0.9631 - val_accuracy: 0.7479
Epoch 404/500
46/46 [==============================] - 0s 2ms/step - loss: 0.2016 - accuracy: 0.9082 - val_loss: 1.0114 - val_accuracy: 0.7068
Epoch 405/500
46/46 [==============================] - 0s 2ms/step - loss: 0.2086 - accuracy: 0.9027 - val_loss: 0.9077 - val_accuracy: 0.7534
Epoch 406/500
46/46 [==============================] - 0s 1ms/step - loss: 0.2119 - accuracy: 0.9089 - val_loss: 1.0139 - val_accuracy: 0.7123
Epoch 407/500
46/46 [==============================] - 0s 2ms/step - loss: 0.2123 - accuracy: 0.9110 - val_loss: 0.9758 - val_accuracy: 0.7123
Epoch 408/500

46/46 [==============================] - 0s 1ms/step - loss: 0.1974 - accuracy: 0.9123 - val_loss: 1.0861 - val_accuracy: 0.7205
Epoch 459/500
46/46 [==============================] - 0s 2ms/step - loss: 0.1810 - accuracy: 0.9212 - val_loss: 1.0984 - val_accuracy: 0.6740
Epoch 460/500
46/46 [==============================] - 0s 2ms/step - loss: 0.1908 - accuracy: 0.9144 - val_loss: 1.0146 - val_accuracy: 0.7342
Epoch 461/500
46/46 [==============================] - 0s 1ms/step - loss: 0.1946 - accuracy: 0.9130 - val_loss: 0.9779 - val_accuracy: 0.7370
Epoch 462/500
46/46 [==============================] - 0s 1ms/step - loss: 0.1881 - accuracy: 0.9205 - val_loss: 1.1384 - val_accuracy: 0.7151
Epoch 463/500
46/46 [==============================] - 0s 1ms/step - loss: 0.1987 - accuracy: 0.9185 - val_loss: 0.9914 - val_accuracy: 0.7370
Epoch 464/500
46/46 [==============================] - 0s 1ms/step - loss: 0.1890 - accuracy: 0.9199 - val_loss: 1.0658 - val_accuracy: 0.7370
Epoch 465/500

In [24]:
y_pred = ann.predict(X_test_ann)
auc = roc_auc_score(y_test_ann, y_pred)
print('AUC: ', auc)

AUC:  0.7875519046759343


# ANN + LSTM + embedding

LSTM (24h) + ANN (media delle features nel giorno precedente) + embedding sul mese

In [25]:
df_prev = df
df_prev.drop(['hour'], axis=1, inplace=True)
df_prev = df.groupby('timestamp').mean()
df_prev.head()

,dew,temp,press,wnd_spd,snow,rain,pollution
timestamp,,,,,,,
2010-01-02,-8.500000,-5.125000,1024.750000,24.860000,0.708333,0.0,1.0
2010-01-03,-10.125000,-8.541667,1022.791667,70.937917,14.166667,0.0,0.0
2010-01-04,-20.875000,-11.500000,1029.291667,111.160833,0.000000,0.0,0.0
2010-01-05,-24.583333,-14.458333,1033.625000,56.920000,0.000000,0.0,0.0
2010-01-06,-23.708333,-12.541667,1033.750000,18.511667,0.000000,0.0,0.0


In [26]:
df_0 = pd.DataFrame({'dew': df['dew'].mean(), 
                    'temp': df['temp'].mean(), 
                    'press': df['press'].mean(),
                    'wnd_spd': df['wnd_spd'].mean(),
                    'snow': df['snow'].mean(),
                    'rain': df['rain'].mean(),
                    'pollution': df['pollution'].mean()}, index=['2010-01-01'])
#df_prev = pd.concat([df_0.head(), df_prev], axis=0)
df_prev = pd.concat([df_0, df_prev], axis=0)
df_prev.head()

,dew,temp,press,wnd_spd,snow,rain,pollution
2010-01-01,1.828516,12.459041,1016.447306,23.894307,0.052763,0.195023,0.481096
2010-01-02,-8.500000,-5.125000,1024.750000,24.860000,0.708333,0.000000,1.000000
2010-01-03,-10.125000,-8.541667,1022.791667,70.937917,14.166667,0.000000,0.000000
2010-01-04,-20.875000,-11.500000,1029.291667,111.160833,0.000000,0.000000,0.000000
2010-01-05,-24.583333,-14.458333,1033.625000,56.920000,0.000000,0.000000,0.000000


In [27]:
train_size = int(len(df_prev.index) * 0.8)
X_train_ann = df_prev.iloc[:train_size, :]
X_test_ann = df_prev.iloc[train_size:-1, :]

Added embedding layer on the day

In [28]:
df_day = pd.DataFrame(df_prev.index, columns=['month'])
df_day.index = df_prev.index
df_day['month'] = pd.to_datetime(df_day["month"])
df_day['month'] = df_day.month.dt.strftime('%m')
df_day.head()

,month
2010-01-01,01
2010-01-02,01
2010-01-03,01
2010-01-04,01
2010-01-05,01


In [29]:
df_day['month_int'] = df_day['month'].astype('int32')
df_day.drop('month', axis=1, inplace=True)
df_day

,month_int
2010-01-01,1
2010-01-02,1
2010-01-03,1
2010-01-04,1
2010-01-05,1
...,...
2014-12-27,12
2014-12-28,12
2014-12-29,12
2014-12-30,12


In [30]:
train_size = int(len(df_day.index) * 0.8)
X_train_emb = df_day.iloc[:train_size, :]
X_test_emb = df_day.iloc[train_size:-1, :]

Model

In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Flatten

first_input = Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
rnn_layer1 = LSTM(units=100, return_sequences=True)(first_input)
rnn_layer2 = LSTM(units=100, return_sequences=False)(rnn_layer1)
rnn_layer3 = Dense(20, activation='relu')(rnn_layer2)

second_input = Input(shape=(X_train_ann.shape[1], ))
ann_layer1 = Dense(5, activation='relu')(second_input)

third_input = Input(shape=(1,1))
emb_layer1 = Embedding(13, 1)(third_input)
emb_layer2 = Flatten()(emb_layer1)

merged = concatenate([rnn_layer3, ann_layer1, emb_layer2])
dense = Dense(10, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[first_input, second_input, third_input], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

model.fit([X_train_lstm, X_train_ann, X_train_emb], y_train_lstm, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
46/46 [==============================] - 4s 19ms/step - loss: 106.1331 - auc: 0.5000
Epoch 2/100
46/46 [==============================] - 1s 19ms/step - loss: 40.7637 - auc: 0.5000
Epoch 3/100
46/46 [==============================] - 1s 19ms/step - loss: 3.5378 - auc: 0.5461
Epoch 4/100
46/46 [==============================] - 1s 19ms/step - loss: 0.7016 - auc: 0.6736
Epoch 5/100
46/46 [==============================] - 1s 19ms/step - loss: 0.6744 - auc: 0.6888
Epoch 6/100
46/46 [==============================] - 1s 20ms/step - loss: 0.6798 - auc: 0.6686
Epoch 7/100
46/46 [==============================] - 1s 21ms/step - loss: 0.6597 - auc: 0.6901
Epoch 8/100
46/46 [==============================] - 1s 20ms/step - loss: 0.6476 - auc: 0.6916
Epoch 9/100
46/46 [==============================] - 1s 19ms/step - loss: 0.6609 - auc: 0.6664
Epoch 10/100
46/46 [==============================] - 1s 19ms/step - loss: 0.6406 - auc: 0.6919
Epoch 11/100
46/46 [==========================

46/46 [==============================] - 1s 22ms/step - loss: 0.6076 - auc: 0.7238
Epoch 87/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6164 - auc: 0.7118
Epoch 88/100
46/46 [==============================] - 1s 23ms/step - loss: 0.6035 - auc: 0.7319
Epoch 89/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6479 - auc: 0.6818
Epoch 90/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6301 - auc: 0.6998
Epoch 91/100
46/46 [==============================] - 1s 23ms/step - loss: 0.6013 - auc: 0.7343
Epoch 92/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6070 - auc: 0.7270
Epoch 93/100
46/46 [==============================] - 1s 22ms/step - loss: 0.5945 - auc: 0.7425
Epoch 94/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6240 - auc: 0.7071
Epoch 95/100
46/46 [==============================] - 1s 22ms/step - loss: 0.6413 - auc: 0.6946
Epoch 96/100
46/46 [==============================] -

In [32]:
from sklearn.metrics import roc_auc_score

y_pred = model.predict([X_test_lstm, X_test_ann, X_test_emb])
auc = roc_auc_score(y_test_lstm, y_pred)
print('AUC: ', auc)

AUC:  0.7115303604742131


In [33]:
for layer in model.layers:
    weights = layer.get_weights()
    if 'Embedding' in str(layer):
        print(weights)

[array([[-0.046547  ],
       [ 0.01539929],
       [ 0.2521121 ],
       [ 0.10881738],
       [ 0.14236435],
       [ 0.14570846],
       [ 0.26986825],
       [-0.3041248 ],
       [-0.63447654],
       [-0.69903916],
       [-0.49912006],
       [ 0.05343078],
       [ 0.05170352]], dtype=float32)]
